In [ ]:
import spotipy
import spotipy.util as util
import pandas as pd
import os
import time
import requests
from bs4 import BeautifulSoup as bs
import datetime
from datetime import timedelta, date
import glob
from os import listdir
import numpy as np
import pymongo
import csv
import json
import sys, getopt, pprint
from pymongo import MongoClient
import types
import itertools
from itertools import zip_longest

In [ ]:
def current_user_recently_played(self, limit=50):
    return self._get('me/player/recently-played', limit=limit)

In [ ]:
scope="user-read-recently-played user-read-private user-top-read user-read-currently-playing"

In [ ]:
if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print ("Usage: %s username" % (sys.argv[0],))
    sys.exit()

In [ ]:
token = util.prompt_for_user_token(username,scope,
                           client_id='e67d7b1bc1ee460bbef7484ee3f601ef',
                           client_secret='ba0087b5f4ab4fb8ab88be1850854538',
                           redirect_uri='https://google.com')

In [ ]:
spotify = spotipy.Spotify(auth=token)
spotify.current_user_recently_played = types.MethodType(current_user_recently_played, spotify)



In [ ]:
dave = spotify.current_user_recently_played(limit=50)
out_file = open("dave.json","w")
out_file.write(json.dumps(dave, sort_keys=True, indent=2))
out_file.close()

In [ ]:
#print(json.dumps(dave, sort_keys=True, indent=2))

In [ ]:
Tracks = []

for i in range(50):
   Tracks.append(dave['items'][i]['track']['name'])

TrackID = []

for i in range(50):
   TrackID.append(dave['items'][i]['track']['id'])

In [ ]:
Artists = []

for i in range(50):
   Artists.append(dave['items'][i]['track']['artists'][0]['name'])

In [ ]:
Albums = []


for i in range(50):
   Albums.append(dave['items'][i]['track']['album']['name'])

In [ ]:
for (a, b, c, d) in itertools.zip_longest(TrackID, Tracks, Artists, Albums): 
    print (a, b, c,d ) 

In [ ]:
dave_df = pd.DataFrame(np.column_stack([TrackID, Tracks, Artists, Albums]), 
                               columns=['Track ID','Tracks', 'Artists', 'Albums'])

In [ ]:
dave_df.head(5)

In [ ]:
TrackID

In [50]:
features = []
for i in range(0,len(TrackID),50):
    audio_features = spotify.audio_features(TrackID[i:i+50])
    for track in audio_features:
        features.append(track)
        features[-1]['target'] = 1

In [71]:
features

[{'danceability': 0.376,
  'energy': 0.43,
  'key': 0,
  'loudness': -6.585,
  'mode': 0,
  'speechiness': 0.0328,
  'acousticness': 0.0902,
  'instrumentalness': 0,
  'liveness': 0.0912,
  'valence': 0.129,
  'tempo': 77.599,
  'type': 'audio_features',
  'id': '4Yxc55NX3tAXC2mHRAhtcW',
  'uri': 'spotify:track:4Yxc55NX3tAXC2mHRAhtcW',
  'track_href': 'https://api.spotify.com/v1/tracks/4Yxc55NX3tAXC2mHRAhtcW',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4Yxc55NX3tAXC2mHRAhtcW',
  'duration_ms': 177323,
  'time_signature': 3,
  'target': 1},
 {'danceability': 0.598,
  'energy': 0.427,
  'key': 7,
  'loudness': -8.764,
  'mode': 0,
  'speechiness': 0.0317,
  'acousticness': 0.0546,
  'instrumentalness': 5.83e-06,
  'liveness': 0.21,
  'valence': 0.0605,
  'tempo': 76.469,
  'type': 'audio_features',
  'id': '3eekarcy7kvN4yt5ZFzltW',
  'uri': 'spotify:track:3eekarcy7kvN4yt5ZFzltW',
  'track_href': 'https://api.spotify.com/v1/tracks/3eekarcy7kvN4yt5ZFzltW',
  'analysis_url

In [60]:
audiofeat_df = pd.DataFrame(features)

In [61]:
audiofeat_df.drop(audiofeat_df.loc[:,'uri':'target'], inplace = True, axis = 1)


In [64]:
del audiofeat_df['type']

In [99]:
audiofeat_df.head(1)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id
0,0.376,0.43,0,-6.585,0,0.0328,0.0902,0.0,0.0912,0.129,77.599,4Yxc55NX3tAXC2mHRAhtcW


In [67]:
DaveDF = dave_df.join(audiofeat_df)

In [69]:
del DaveDF['id']

In [84]:
DaveDF

,Track ID,Tracks,Artists,Albums,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,4Yxc55NX3tAXC2mHRAhtcW,"All I Want - From ""High School Musical: The Mu...",Olivia Rodrigo,"All I Want (From ""High School Musical: The Mus...",0.376,0.430,0,-6.585,0,0.0328,0.090200,0.000000,0.0912,0.1290,77.599
1,3eekarcy7kvN4yt5ZFzltW,HIGHEST IN THE ROOM,Travis Scott,HIGHEST IN THE ROOM,0.598,0.427,7,-8.764,0,0.0317,0.054600,0.000006,0.2100,0.0605,76.469
2,0zyHqmRo3xthdkEZR149or,Threesome,Fenix TX,Lechuza,0.384,0.886,7,-3.053,1,0.1020,0.002360,0.000000,0.1630,0.5360,99.522
3,7qZdLKPA5TXDxpd10AGpf0,Warm Me Up,The Audition,Champion,0.477,0.951,8,-2.499,0,0.0558,0.008700,0.000000,0.1210,0.7540,162.956
4,1iD0a0plhO55cWEJa5UGcZ,The Science of Selling Yourself Short,Less Than Jake,Anthem,0.719,0.815,5,-3.546,1,0.0305,0.160000,0.000000,0.0367,0.9370,93.126
5,63IIUzd6eJBfCGIcF8MFnJ,Way Away,Yellowcard,Ocean Avenue,0.357,0.959,2,-4.081,0,0.1130,0.000007,0.000039,0.0825,0.3640,179.644
6,27O9FuS1UIm6ZyLplgLQAH,Devotion And Desire,Bayside,Bayside,0.296,0.900,2,-4.918,0,0.0922,0.000702,0.000000,0.3390,0.2860,184.256
7,44BrW9G0DSu50xItRb1gFK,Vicious Love (feat. Hayley Williams),New Found Glory,Resurrection: Ascension,0.615,0.907,5,-2.908,1,0.0552,0.008180,0.000027,0.1450,0.7160,109.962
8,4wzjNqjKAKDU82e8uMhzmr,Face Down,The Red Jumpsuit Apparatus,Don't You Fake It,0.545,0.932,7,-2.189,0,0.0399,0.000665,0.000000,0.1270,0.4650,92.956
9,5jZ1Z2GFTf2gwmFc3qiUxs,The Taste of Ink,The Used,The Used (U.S. Version),0.541,0.904,7,-3.001,1,0.0435,0.001430,0.000000,0.1930,0.5410,98.122


In [94]:
#DaveRGL = DaveDF[['Track ID','danceability','energy',
                  'speechiness', 'acousticness',
                  'instrumentalness', 'liveness', 'valence']]

In [98]:
#DaveRGL.head(1)

,Track ID,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence
0,4Yxc55NX3tAXC2mHRAhtcW,0.376,0.43,0.0328,0.0902,0.0,0.0912,0.129


In [97]:
#DaveRGL.to_csv('DaveRGL.csv',index=False)